# Notebook for testing app code

In [75]:
import joblib
import pandas as pd
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

# app = FastAPI()

# app.add_middleware(
#     CORSMiddleware,
#     allow_origins=["*"],  # replace with your frontend URL for production
#     allow_credentials=True,
#     allow_methods=["*"],
#     allow_headers=["*"],
# )

class DataHandler:
    def __init__(self):
        self.data: pd.DataFrame = joblib.load("C:/Code/Git Repositories/Football/Football/3_Data_Preparation/rawdata_clean.pkl")

    def team_query(self, home_team: str, away_team: str) -> pd.DataFrame:

        home_team_stats = self.data[self.data['home_team'] == home_team].sort_values(by=['season', 'week'], ascending=False).iloc[:3]
        home_team_stats = home_team_stats.loc[:, 'season':'hlb3_interception_yards']
        away_team_stats = self.data[self.data['away_team'] == away_team].sort_values(by=['season', 'week'], ascending=False).iloc[:3]
        away_team_stats = away_team_stats.loc[:, 'aqb1':'alb3_interception_yards']

        # home_team_stats = self.data[self.data['home_team'] == home_team].sort_values(by=['season', 'week'], ascending=False).iloc[0]
        # home_team_stats = home_team_stats.loc['season':'hlb3_interception_yards']
        # away_team_stats = self.data[self.data['away_team'] == away_team].sort_values(by=['season', 'week'], ascending=False).iloc[0]
        # away_team_stats = away_team_stats.loc['aqb1':'alb3_interception_yards']

        combined_dict = {}
        away_team_stats_mean = away_team_stats.select_dtypes(include=['float64', 'int64']).mean(axis=0)
        home_team_stats_mean = home_team_stats.select_dtypes(include=['float64', 'int64']).mean(axis=0)
        # Prefix home stats

        for col, val in home_team_stats_mean.items():
            combined_dict[f"{col}"] = val

        # Prefix away stats
        for col, val in away_team_stats_mean.items():
            combined_dict[f"{col}"] = val

        # Create a single-row DataFrame with correct types
        combined_stats = pd.DataFrame([combined_dict])

        print(combined_stats.head())
        return combined_stats

    
    def preprocess_stats(self, game_stats: pd.DataFrame):

        game_stats_processed = game_stats.select_dtypes(include=['float64', 'int64'])

        return game_stats_processed
    

class ModelHandler:
    def __init__(self):
        self.lasso_pipe = joblib.load("C:/Code/Git Repositories/Football/Football/4_5_Modeling_and_Evaluation/lasso_pipeline.pkl")

    def make_prediction(self, home_team: str, away_team: str):
        query = DataHandler()

        game_stats = query.team_query(home_team=home_team, away_team=away_team)
        game_stats_processed = query.preprocess_stats(game_stats)
        prediction = self.lasso_pipe.predict(X=game_stats_processed)
        return prediction[0]


# Define input data model for API
class Teams(BaseModel):
    home_team: str
    away_team: str

model = ModelHandler()

# @app.post("/predict")
def predict_margin(teams: Teams):
    print(f"Received request: {teams.home_team} vs {teams.away_team}")
    pred = model.make_prediction(teams.home_team, teams.away_team)

    if pred > 0:
        winner = teams.home_team
    else:
        winner = teams.away_team
    print(f"Prediction: {pred}, Winner: {winner}")
    return {"predicted_margin": pred, "predicted_winner": winner}

def main():
    query = DataHandler()

    return query.team_query('DAL', 'BAL')




In [76]:
main()

   season  week  spread_line  hqb1_passing_epa  hqb1_completions  \
0  2024.0  16.0         -5.5          0.244109         20.666667   

   hqb1_attempts  hqb1_passing_yards  hqb1_passing_tds  hqb1_interceptions  \
0      33.333333          239.666667               1.0            0.333333   

   hqb1_sacks  ...  alb3_tackles_solo  alb3_tackles_with_assist  \
0         2.0  ...                2.0                       0.0   

   alb3_tackles_for_loss  alb3_tackles_for_loss_yards  alb3_fumbles_forced  \
0               0.666667                          0.0                  0.0   

   alb3_sacks  alb3_sack_yards  alb3_qb_hits  alb3_interceptions  \
0    0.666667         4.333333           1.0                 0.0   

   alb3_interception_yards  
0                      0.0  

[1 rows x 331 columns]


,season,week,spread_line,hqb1_passing_epa,hqb1_completions,hqb1_attempts,hqb1_passing_yards,hqb1_passing_tds,hqb1_interceptions,hqb1_sacks,...,alb3_tackles_solo,alb3_tackles_with_assist,alb3_tackles_for_loss,alb3_tackles_for_loss_yards,alb3_fumbles_forced,alb3_sacks,alb3_sack_yards,alb3_qb_hits,alb3_interceptions,alb3_interception_yards
0,2024.0,16.0,-5.5,0.244109,20.666667,33.333333,239.666667,1.0,0.333333,2.0,...,2.0,0.0,0.666667,0.0,0.0,0.666667,4.333333,1.0,0.0,0.0
